In [2]:
import pandas as pd
from scipy.fftpack import fft
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as s
import os


# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

Using TensorFlow backend.


In [14]:
data = pd.read_csv('LSTM_mag.csv')
print(data.head())

            File    t_1    t_2    t_3    t_4    t_5    t_6    t_7    t_8  \
0  no_act_1(0,1)  14.04  12.65  12.65  12.21  12.08  12.21  13.15  12.21   
1  no_act_1(0,2)  12.81  13.00  13.04  13.60  12.00  12.04  13.60  11.70   
2  no_act_1(0,3)  13.04  13.34  13.04  13.60  13.60  15.13  13.45  13.00   
3  no_act_1(0,4)  13.04  13.34  13.00  13.60  12.53  13.93  12.21  13.60   
4  no_act_1(0,5)  12.65  13.15  14.42  12.53  12.04  12.65  13.04  13.00   

     t_9  ...  t_292  t_293  t_294  t_295  t_296  t_297  t_298  t_299  t_300  \
0  13.04  ...  12.21  13.15  13.15  13.00  13.00  12.21  15.03  13.04  12.65   
1  13.04  ...  13.45  13.00  14.00  13.60  13.93  13.04  12.08  12.21  13.60   
2  12.65  ...  13.15  13.00  13.60  13.04  12.21  13.00  12.04  12.53  12.37   
3  13.93  ...  13.42  13.15  13.60  12.81  13.60  13.34  13.04  12.21  12.53   
4  13.15  ...  14.04  13.60  12.04  12.65  13.60  13.15  14.87  13.60  12.81   

     Y  
0  0.0  
1  0.0  
2  0.0  
3  0.0  
4  0.0  

[5 rows

In [79]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
train, test = train_test_split(data, test_size=.1)

In [80]:
trainX = np.array(train[train.columns[1:301]])
trainX = np.reshape(trainX,(trainX.shape[0],trainX.shape[1],1))
trainY = train["Y"]

testX = np.array(test[test.columns[1:301]])
testX = np.reshape(testX,(testX.shape[0],testX.shape[1],1))
testY = test["Y"]

print(trainX.shape,trainY.shape)
print(testX.shape,testY.shape)

trainY = to_categorical(trainY)
testY = to_categorical(testY)
print(trainX.shape,trainY.shape)
print(testX.shape,testY.shape)

(10602, 300, 1) (10602,)
(1178, 300, 1) (1178,)
(10602, 300, 1) (10602, 2)
(1178, 300, 1) (1178, 2)


In [81]:
def make_model(trainX,trainY,testX,testY):
    verbose , epochs, bs= 1,7,2000
    n_timesteps,n_features,n_outputs = trainX.shape[1],trainX.shape[2], testY.shape[1]
    
    
    model=Sequential()
    model.add(LSTM(100,input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100,activation= 'relu'))
    model.add(Dense(n_outputs,activation = 'softmax'))
    
    model.compile(loss = 'categorical_crossentropy',optimizer= 'adam',metrics = ['accuracy'])
    model.fit(trainX,trainY,epochs= epochs,batch_size = bs,verbose = 1)
    
    _,accuracy  = model.evaluate(testX,testY,batch_size = bs,verbose = 1)
    print(model.summary())
    return accuracy
    
    

In [82]:
def run_expt(trainX,trainY,testX,testY,repeats=2):
    scores = list()
    for r in range(repeats):
        score = make_model(trainX,trainY,testX,testY)
        score = score*100
        print('>> #%d: %.3f' %(r+1,score))
        scores.append(score)
    print(scores)
run_expt(trainX,trainY,testX,testY,repeats=3)

Epoch 1/7
10602/10602 [==============================] - 4s 360us/step - loss: 0.3015 - accuracy: 0.9401
Epoch 2/7
10602/10602 [==============================] - 3s 319us/step - loss: 0.1585 - accuracy: 0.9681
Epoch 3/7
10602/10602 [==============================] - 3s 319us/step - loss: 0.1660 - accuracy: 0.9681
Epoch 4/7
10602/10602 [==============================] - 3s 321us/step - loss: 0.1525 - accuracy: 0.9681
Epoch 5/7
10602/10602 [==============================] - 3s 327us/step - loss: 0.1412 - accuracy: 0.9681
Epoch 6/7
10602/10602 [==============================] - 3s 327us/step - loss: 0.1404 - accuracy: 0.9681
Epoch 7/7
1178/1178 [==============================] - 0s 151us/step
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 100)               40800     
_________________________________________________________________
dropout_18 (Dr